In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['PR']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('progesterone receptor', 2204),
 ('pathogenesis related', 326),
 ('protease', 314),
 ('partial response', 176),
 ('progesterone', 118),
 ('progressive ratio', 116),
 ('pulmonary rehabilitation', 77),
 ('photoreceptor', 71),
 ('primary root', 58),
 ('progestin receptor', 48),
 ('partial remission', 42),
 ('pulmonary regurgitation', 42),
 ('protein restriction', 37),
 ('prolonged release', 33),
 ('production rate', 33),
 ('proteorhodopsin', 29),
 ('pregnancy rate', 28),
 ('pulse rate', 28),
 ('platelet reactivity', 27),
 ('protein', 22),
 ('proline rich', 20),
 ('puerarin', 16),
 ('p4 receptor', 16),
 ('progestin', 14),
 ('pityriasis rosea', 14),
 ('pathogen related', 14),
 ('pathogen reduction', 12),
 ('prevalence ratio', 12),
 ('paclitaxel resistance', 12),
 ('placental restriction', 11),
 ('polymerization rate', 11),
 ('positive remodeling', 10),
 ('platelet release', 10),
 ('partial', 10),
 ('polyrotaxanes', 10),
 ('photoreactivation', 10),
 ('puerto rico', 9),
 ('phosphate rock', 

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-06 15:17:50] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'p receptor': 'HGNC:8910',
  'p4 receptor': 'HGNC:8910',
  'paclitaxel resistance': 'MESH:D019008',
  'paeoniae radix': 'ungrounded',
  'pain relief': 'ungrounded',
  'palindromic rheumatism': 'MESH:C538103',
  'papaver rhoeas': 'ungrounded',
  'paradoxical reaction': 'ungrounded',
  'paradoxical response': 'ungrounded',
  'parenchymal rejection': 'ungrounded',
  'parmigiano reggiano': 'ungrounded',
  'partial': 'NCIT:C18058',
  'partial regression': 'ungrounded',
  'partial remission': 'NCIT:C18058',
  'partial response': 'NCIT:C18058',
  'particle radioactivity': 'MESH:D011851',
  'pathogen reduction': 'ungrounded',
  'pathogen related': 'pathogenesis_related',
  'pathogenesis related': 'pathogenesis_related',
  'pathological response': 'ungrounded',
  'patlak rutland method': 'MESH:D008722',
  'peer reared': 'ungrounded',
  'peg ifn rbv': 'CHEBI:CHEBI:63580',
  'peginterferon and ribavirin': 'CHEBI:CHEBI:63580',
  'pegylated interferon and ribavirin': 'CHEBI:CHEBI:63580',
  'penet

In [7]:
grounding_map, names, pos_labels = [{'p receptor': 'HGNC:8910',
  'p4 receptor': 'HGNC:8910',
  'paclitaxel resistance': 'MESH:D019008',
  'paeoniae radix': 'ungrounded',
  'pain relief': 'ungrounded',
  'palindromic rheumatism': 'MESH:C538103',
  'papaver rhoeas': 'ungrounded',
  'paradoxical reaction': 'ungrounded',
  'paradoxical response': 'ungrounded',
  'parenchymal rejection': 'ungrounded',
  'parmigiano reggiano': 'ungrounded',
  'partial': 'NCIT:C18058',
  'partial regression': 'ungrounded',
  'partial remission': 'NCIT:C18058',
  'partial response': 'NCIT:C18058',
  'particle radioactivity': 'MESH:D011851',
  'pathogen reduction': 'ungrounded',
  'pathogen related': 'pathogenesis_related',
  'pathogenesis related': 'pathogenesis_related',
  'pathological response': 'ungrounded',
  'patlak rutland method': 'ungrounded',
  'peer reared': 'ungrounded',
  'peg ifn rbv': 'CHEBI:CHEBI:63580',
  'peginterferon and ribavirin': 'CHEBI:CHEBI:63580',
  'pegylated interferon and ribavirin': 'CHEBI:CHEBI:63580',
  'penetration resistance': 'ungrounded',
  'peptide receptivity': 'ungrounded',
  'perceived restorativeness': 'ungrounded',
  'perfusion rate': 'ungrounded',
  'perichromatin region': 'ungrounded',
  'peripheral resistance': 'ungrounded',
  'perirhinal cortex': 'MESH:D000071039',
  'persistent remission': 'EFO:0009785',
  'pgr': 'HGNC:8910',
  'phenol red': 'CHEBI:CHEBI:31991',
  'phenylethyl resorcinol': 'MESH:C512391',
  'philosamia ricini': 'ungrounded',
  'phosphate rock': 'ungrounded',
  'phosphoramidon': 'CHEBI:CHEBI:45353',
  'phosphorus removal': 'ungrounded',
  'phosphorylase b': 'FPLX:PYG',
  'photoreactivation': 'ungrounded',
  'photoreceptor': 'MESH:D010786',
  'photorefractory': 'ungrounded',
  'photoresist': 'ungrounded',
  'phytolaccae radix': 'ungrounded',
  'pityriasis rosea': 'MESH:D017515',
  'placental restriction': 'ungrounded',
  'platelet reactivity': 'ungrounded',
  'platelet release': 'ungrounded',
  'plus ribavirin': 'CHEBI:CHEBI:63580',
  'pml rarα': 'ungrounded',
  'pneumatic retinopexy': 'ungrounded',
  'polygalae radix': 'ungrounded',
  'polymerization rate': 'ungrounded',
  'polyrotaxanes': 'MESH:D043862',
  'positive remodeling': 'ungrounded',
  'pr interval': 'EFO:0004462',
  'pragmatic rehabilitation': 'ungrounded',
  'precision recall': 'ungrounded',
  'prednisolone': 'CHEBI:CHEBI:8378',
  'pregnancy rate': 'MESH:D018873',
  'prepotent response': 'ungrounded',
  'prepster ™': 'ungrounded',
  'present rate': 'ungrounded',
  'prevalence ratio': 'ungrounded',
  'primary realignment': 'ungrounded',
  'primary response': 'ungrounded',
  'primary root': 'primary_root',
  'probe': 'ungrounded',
  'process': 'ungrounded',
  'production rate': 'production_rate',
  'progesterone': 'HGNC:8910',
  'progesterone receptor': 'HGNC:8910',
  'progestin': 'HGNC:8910',
  'progestin receptor': 'HGNC:8910',
  'progestogen receptor': 'HGNC:8910',
  'program': 'ungrounded',
  'progressive ratio': 'progressive_ratio',
  'proliferative response': 'ungrounded',
  'proliferative retinopathy': 'MESH:D012164',
  'proline reductase': 'ungrounded',
  'proline rich': 'ungrounded',
  'prolonged release': 'MESH:D003692',
  'promoter': 'ungrounded',
  'propranolol': 'CHEBI:CHEBI:8499',
  'prorenin': 'ungrounded',
  'protease': 'FPLX:Protease',
  'protein': 'CHEBI:CHEBI:36080',
  'protein restriction': 'MESH:D018753',
  'proteinase': 'MESH:D010450',
  'proteorhodopsin': 'IP:IPR017402',
  'proton radiation': 'MESH:D011827',
  'proximal region': 'ungrounded',
  'pseudorabies': 'MESH:D011557',
  'puerariae radix': 'ungrounded',
  'puerarin': 'CHEBI:CHEBI:8633',
  'puerto rico': 'ungrounded',
  'pulmonary regurgitation': 'MESH:D011665',
  'pulmonary rehabilitation': 'NCIT:C157966',
  'pulmonary response': 'ungrounded',
  'pulp residues': 'ungrounded',
  'pulse rate': 'MESH:D006339',
  'purinergic receptor': 'FPLX:Purinergic_receptors',
  'putative resistance': 'ungrounded',
  'pyrogallol red': 'MESH:C058731',
  'spectra of r12': 'HGNC:4471'},
 {'HGNC:8910': 'PGR',
  'MESH:D019008': 'Drug Resistance, Neoplasm',
  'MESH:C538103': 'Palindromic rheumatism',
  'NCIT:C18058': 'Partial Remission',
  'MESH:D011851': 'Radioactivity',
  'pathogenesis_related': 'pathogenesis_related',
  'CHEBI:CHEBI:63580': 'ribavirin',
  'MESH:D000071039': 'Perirhinal Cortex',
  'EFO:0009785': 'remission',
  'CHEBI:CHEBI:31991': 'phenol red',
  'MESH:C512391': '4-(1-phenylethyl)-1,3-benzenediol',
  'CHEBI:CHEBI:45353': 'phosphoramidon',
  'FPLX:PYG': 'PYG',
  'MESH:D010786': 'Photoreceptor Cells',
  'MESH:D017515': 'Pityriasis Rosea',
  'MESH:D043862': 'Rotaxanes',
  'EFO:0004462': 'PR interval',
  'CHEBI:CHEBI:8378': 'prednisolone',
  'MESH:D018873': 'Pregnancy Rate',
  'primary_root': 'primary_root',
  'production_rate': 'production_rate',
  'progressive_ratio': 'progressive_ratio',
  'MESH:D012164': 'Retinal Diseases',
  'MESH:D003692': 'Delayed-Action Preparations',
  'CHEBI:CHEBI:8499': 'propranolol',
  'FPLX:Protease': 'Protease',
  'CHEBI:CHEBI:36080': 'protein',
  'MESH:D018753': 'Diet, Protein-Restricted',
  'MESH:D010450': 'Endopeptidases',
  'IP:IPR017402': 'Proteorhodopsin',
  'MESH:D011827': 'Radiation',
  'MESH:D011557': 'Pseudorabies',
  'CHEBI:CHEBI:8633': 'puerarin',
  'MESH:D011665': 'Pulmonary Valve Insufficiency',
  'NCIT:C157966': 'Pulmonary Rehabilitation',
  'MESH:D006339': 'Heart Rate',
  'FPLX:Purinergic_receptors': 'Purinergic_receptors',
  'MESH:C058731': 'pyrogallol sulfonphthalein',
  'HGNC:4471': 'GPR17'},
 ['FPLX:Protease',
  'HGNC:8910',
  'IP:IPR017402',
  'MESH:D006339',
  'MESH:D010786',
  'MESH:D011665',
  'MESH:D018753',
  'MESH:D018873',
  'NCIT:C157966',
  'NCIT:C18058']]

In [8]:
excluded_longforms = []

In [9]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [10]:
additional_entities = {}

In [11]:
unambiguous_agent_texts = {}

In [12]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [15]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [16]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [17]:
intersection1

[('HGNC:17981', 'HGNC:17981', 29)]

In [18]:
intersection2

[('ungrounded', 'HGNC:17981', 0)]

In [20]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['RTCA', 'RTCD1', 'RPC', 'RTC1', 'RTC'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [ ]:
names.update(additional_entitie)

In [13]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-22 05:20:06] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-22 05:22:33] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9606602392407335 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [16]:
classifier.stats

{'label_distribution': {'HGNC:8910': 1834,
  'production_rate': 27,
  'CHEBI:CHEBI:8499': 4,
  'ungrounded': 206,
  'NCIT:C18058': 196,
  'MESH:D010450': 7,
  'MESH:C538103': 4,
  'CHEBI:CHEBI:31991': 7,
  'FPLX:Protease': 221,
  'MESH:D010786': 54,
  'MESH:D018873': 23,
  'MESH:D012164': 4,
  'MESH:D018753': 19,
  'CHEBI:CHEBI:8378': 3,
  'MESH:D006339': 23,
  'CHEBI:CHEBI:36080': 20,
  'CHEBI:CHEBI:45353': 3,
  'pathogenesis_related': 272,
  'CHEBI:CHEBI:50406': 1,
  'NCIT:C157966': 45,
  'progressive_ratio': 82,
  'MESH:C058731': 4,
  'IP:IPR017402': 22,
  'MESH:D017515': 10,
  'MESH:D011557': 5,
  'FPLX:PYG': 1,
  'MESH:D011665': 29,
  'MESH:D003692': 17,
  'MESH:D011827': 1,
  'MESH:D008722': 1,
  'MESH:D000071039': 2,
  'CHEBI:CHEBI:8633': 15,
  'primary_root': 30,
  'MESH:D043862': 8,
  'CHEBI:CHEBI:63580': 10,
  'EFO:0004462': 2,
  'MESH:D019008': 7,
  'MESH:D019542': 3,
  'EFO:0009785': 2,
  'MESH:C512391': 3,
  'FPLX:Purinergic_receptors': 1,
  'MESH:D011851': 3,
  'HGNC:4471

In [14]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [15]:
disamb.dump(model_name, results_path)

In [16]:
print(disamb.info())

Disambiguation model for PR

Produces the disambiguations:
	4-(1-phenylethyl)-1,3-benzenediol	MESH:C512391
	Delayed-Action Preparations	MESH:D003692
	Diet, Protein-Restricted*	MESH:D018753
	Drug Resistance, Neoplasm	MESH:D019008
	Endopeptidases	MESH:D010450
	GPR17	HGNC:4471
	Heart Rate*	MESH:D006339
	PGR*	HGNC:8910
	PR interval	EFO:0004462
	PYG	FPLX:PYG
	Palindromic rheumatism	MESH:C538103
	Partial Remission*	NCIT:C18058
	Perirhinal Cortex	MESH:D000071039
	Photoreceptor Cells*	MESH:D010786
	Pityriasis Rosea	MESH:D017515
	Pregnancy Rate*	MESH:D018873
	Protease*	FPLX:Protease
	Proteorhodopsin*	IP:IPR017402
	Pseudorabies	MESH:D011557
	Pulmonary Rehabilitation*	NCIT:C157966
	Pulmonary Valve Insufficiency*	MESH:D011665
	Purinergic_receptors	FPLX:Purinergic_receptors
	Radiation	MESH:D011827
	Radioactivity	MESH:D011851
	Retinal Diseases	MESH:D012164
	Rotaxanes	MESH:D043862
	pathogenesis_related	pathogenesis_related
	phenol red	CHEBI:CHEBI:31991
	phosphoramidon	CHEBI:CHEBI:45353
	prednisolone	

In [17]:
model_to_s3(disamb)